In [1]:
# !pip install catboost
# !pip install category_encoders
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score, f1_score,jaccard_score,multilabel_confusion_matrix,log_loss
import lightgbm as lgbm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv('F:/data/adult/adult.data',header=None)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [3]:
#encodings and split

def onehot_all(X,y,ratio):
    #convert y to binary
    y=pd.Series(ce.OrdinalEncoder(return_df=0).fit_transform(y).ravel()-1)
    #split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=42)
    
    obj_cols=X.select_dtypes('object').columns
    enc=ce.OneHotEncoder(cols=obj_cols,handle_missing='return_nan').fit(X_train,y_train)
    X_train=enc.transform(X_train)
    X_test=enc.transform(X_test)
    
    return X_train, X_test,y_train, y_test

def target_all(X,y,ratio):
    #convert y to binary
    y=pd.Series(ce.OrdinalEncoder(return_df=0).fit_transform(y).ravel()-1)
    #split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=42)
    
#     obj_cols=X.select_dtypes('object').columns
    enc=ce.TargetEncoder(handle_missing='return_nan').fit(X_train,y_train)
    X_train=enc.transform(X_train)
    X_test=enc.transform(X_test)
    
    return X_train,X_test,y_train, y_test
    
def onehot_target(X,y,ratio,thresh):
    #convert y to binary
    y=pd.Series(ce.OrdinalEncoder(return_df=0).fit_transform(y).ravel()-1)
    
    #split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio, random_state=42)
    
    low_card_cols,high_card_cols=[],[]
    obj_cols=X.select_dtypes('object').columns
    for col in obj_cols:
        if X_train[col].nunique()<=thresh:
            low_card_cols.append(col)
        else:
            high_card_cols.append(col)
    
    
    enc=ce.OneHotEncoder(cols=low_card_cols,handle_missing='return_nan').fit(X_train,y_train)
    X_train=enc.transform(X_train)
    X_test=enc.transform(X_test)
    
    enc=ce.TargetEncoder(cols=high_card_cols,handle_missing='return_nan').fit(X_train,y_train)
    X_train=enc.transform(X_train)
    X_test=enc.transform(X_test)
    
    return X_train,X_test,y_train, y_test
            

In [4]:

def train_evaluate( X_train,X_test,y_train, y_test):
    print("lightgbm training with gridsearch")

    model = lgbm.LGBMClassifier(boosting_type='goss')
    grid = GridSearchCV(estimator=model, param_grid = parameters)
    grid.fit(X_train, y_train,verbose=0)    

    model=grid.best_estimator_
    evaluate(model,X_test,y_test)
    
    print("catboost training with gridsearch")

    model = CatBoostClassifier(
#         task_type="GPU", devices='0:1'
    )
    grid = GridSearchCV(estimator=model, param_grid = parameters)
    grid.fit(X_train, y_train,verbose=0)    

    model=grid.best_estimator_
    evaluate(model,X_test,y_test)


    print("xgboost training with gridsearch")

    model = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
    grid = GridSearchCV(estimator=model, param_grid = parameters)
    grid.fit(X_train, y_train,verbose=0)    

    model=grid.best_estimator_
    evaluate(model,X_test,y_test)

In [5]:
#evaluation function

def evaluate(model,X_test,y_test):
    
    pred=model.predict(X_test)
    pred_proba=model.predict_proba(X_test)

    print('accuracy:',accuracy_score(y_test,pred))
    print('f1 macro:',f1_score(y_test,pred, average='macro'))
    print('f1_micro:',f1_score(y_test,pred, average='micro'))
    print('log_loss:',log_loss(y_test,pred_proba))



In [6]:
# print("catboost training")
# model_cat = CatBoostClassifier(iterations=10,max_depth=10, task_type="GPU", devices='0:1')
# model_cat.fit(X_train, y_train, verbose=0)

#evaluate
# evaluate(model_cat,X_test,y_test)

# print("lightgbm training")

# model_lgbm = lgbm.LGBMClassifier(boosting_type='goss',max_depth=10,n_estimators=10)
# model_lgbm.fit(X_train, y_train)

#evaluate
# evaluate(model_lgbm,X_test,y_test)

# print("xgboost training")

# model_xgb = XGBClassifier(max_depth=10,n_estimators=10,tree_method='gpu_hist', gpu_id=0)
# model_xgb.fit(X_train, y_train)

#evaluate
# evaluate(model_xgb,X_test,y_test)

In [7]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1:]
# X_train,X_test,y_train, y_test=onehot_all(df.iloc[:,:-1],y,.2)
# X_train,X_test,y_train, y_test=target_all(df.iloc[:,:-1],y,ratio=.2)
# X_train,X_test,y_train, y_test=onehot_target(df.iloc[:,:-1],y,ratio=.2,thresh=10)


# parameters = {'depth'         : [6,8,10,12],
#               'learning_rate' : [.01,.05,.1,.2],
#               'iterations'    : [100,500,1000]
#               }
parameters = {'depth'         : [6],
              'learning_rate' : [.2],
              'iterations'    : [100]
              }

print("first with one hot for all")
# X_train,X_test,y_train, y_test=onehot_all(X,y,.2)
# train_evaluate( X_train,X_test,y_train, y_test)

print("now with target encoding")
X_train,X_test,y_train, y_test=target_all(X,y,ratio=.2)
train_evaluate( X_train,X_test,y_train, y_test)

print("now with mix encodings")
X_train,X_test,y_train, y_test=onehot_target(X,y,ratio=.2,thresh=10)
train_evaluate( X_train,X_test,y_train, y_test)


first with one hot for all
now with target encoding
lightgbm training with gridsearch
accuracy: 0.8615077537233226
f1 macro: 0.8049567531603459
f1_micro: 0.8615077537233226
log_loss: 0.3074330304559472
catboost training with gridsearch
accuracy: 0.8762475049900199
f1 macro: 0.8213238543643929
f1_micro: 0.8762475049900199
log_loss: 0.2776349631275468
xgboost training with gridsearch
accuracy: 0.8767081222171043
f1 macro: 0.822986502986503
f1_micro: 0.8767081222171044
log_loss: 0.27315083376994603
now with mix encodings
lightgbm training with gridsearch
accuracy: 0.8602794411177644
f1 macro: 0.8042266973870336
f1_micro: 0.8602794411177644
log_loss: 0.3283249482488637
catboost training with gridsearch
accuracy: 0.8773222785198833
f1 macro: 0.8236110005266362
f1_micro: 0.8773222785198833
log_loss: 0.27406890029165804
xgboost training with gridsearch
accuracy: 0.8800859818823891
f1 macro: 0.8280031931980513
f1_micro: 0.8800859818823891
log_loss: 0.2732258607916863
